### BU MODEL kaggle 6.0329076480 RMSE İLE SUBMİSSİON ÜRETİYOR, CatBoost için en iyi RMSE: 5.447921226050399

In [3]:
# Gerekli kütüphaneleri import edelim
import pandas as pd
import numpy as np
from datetime import datetime
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import RandomizedSearchCV, KFold
from catboost import CatBoostRegressor

import warnings
warnings.filterwarnings('ignore')

print("1. Veri Setini Yükleme")
# 1. Veri Setini Yükleme
train_df = pd.read_csv('train.csv')
test_df = pd.read_csv('test_x.csv')

print("2. Veri Ön İşleme ve Temizleme")

# a. Yaş hesaplama ve 'Yas' sütununu ekleme
print(" - Yaş hesaplanıyor")
def calculate_age(born, application_year):
    try:
        born = pd.to_datetime(born, errors='coerce', infer_datetime_format=True)
        age = application_year - born.year
        return int(age)
    except:
        return np.nan

# Yaşları hesaplayalım
train_df['Yas'] = train_df.apply(lambda row: calculate_age(row['Dogum Tarihi'], row['Basvuru Yili']), axis=1)
test_df['Yas'] = test_df.apply(lambda row: calculate_age(row['Dogum Tarihi'], row['Basvuru Yili']), axis=1)

# Üniversite okuma sayısını hesaplama
print(" - Üniversite okuma sayısı hesaplanıyor")
def calculate_university_count(row):
    if pd.isna(row['Daha Önceden Mezun Olunduysa, Mezun Olunan Üniversite']):
        return 1  # Sadece şu anki üniversite var
    else:
        return 2  # Hem şu anki üniversite hem de önceki var

# Yeni sütunu ekleyelim
train_df['Universite Okuma Sayisi'] = train_df.apply(calculate_university_count, axis=1)
test_df['Universite Okuma Sayisi'] = test_df.apply(calculate_university_count, axis=1)

# b. Gereksiz sütunların çıkarılması
print(" - Gereksiz sütunlar çıkarılıyor")
columns_to_drop = ['Dogum Tarihi', 'Lise Adi', 'Lise Adi Diger', 'Lise Bolum Diger', 'Daha Önceden Mezun Olunduysa, Mezun Olunan Üniversite']
train_df.drop(columns=columns_to_drop, inplace=True)
test_df.drop(columns=columns_to_drop, inplace=True)

# c. Binary sütunların kodlanması
print(" - Binary sütunlar kodlanıyor")
binary_columns = [
    'Burs Aliyor mu?', 'Daha Once Baska Bir Universiteden Mezun Olmus',
    'Baska Bir Kurumdan Burs Aliyor mu?', 'Girisimcilik Kulupleri Tarzi Bir Kulube Uye misiniz?',
    'Profesyonel Bir Spor Daliyla Mesgul musunuz?', 'Aktif olarak bir STK üyesi misiniz?',
    'Stk Projesine Katildiniz Mi?', 'Girisimcilikle Ilgili Deneyiminiz Var Mi?', 'Ingilizce Biliyor musunuz?'
]

for col in binary_columns:
    for df in [train_df, test_df]:
        df[col] = df[col].astype(str).str.strip().str.lower().fillna('bilinmiyor').map({'evet': 1, 'hayır': 0})

# 'Cinsiyet' sütunu için kodlama
print(" - Cinsiyet sütunu kodlanıyor")
for df in [train_df, test_df]:
    df['Cinsiyet'] = df['Cinsiyet'].astype(str).str.strip().str.lower().fillna('bilinmiyor').map({'erkek': 2, 'kadın': 1, 'belirtmek istemiyorum': 0})

# 'Universite Turu' sütunu için kodlama
print(" - Üniversite Türü sütunu kodlanıyor")
for df in [train_df, test_df]:
    df['Universite Turu'] = df['Universite Turu'].astype(str).str.strip().str.lower().fillna('bilinmiyor').map({'özel': 1, 'devlet': 0})

# d. Eksik değerlerin doldurulması
print(" - Eksik değerler dolduruluyor")
# Kategorik sütunları belirleme
categorical_columns = train_df.select_dtypes(include=['object']).columns.tolist()

# Sayısal ve kategorik sütunları ayırma
numerical_columns = train_df.select_dtypes(include=['int64', 'float64']).columns.tolist()
if 'Degerlendirme Puani' in numerical_columns:
    numerical_columns.remove('Degerlendirme Puani')  # Hedef değişkeni çıkardık

# Eğitim verisindeki eksik değerleri dolduralım
for df in [train_df, test_df]:
    for col in numerical_columns:
        median = train_df[col].median()
        df[col].fillna(median, inplace=True)
    for col in categorical_columns:
        df[col] = df[col].astype(str).fillna('Bilinmiyor')

# Hedef değişkende eksik değerleri kontrol edelim ve kaldıralım
print(" - Hedef değişkende eksik değerler kontrol ediliyor")
initial_length = len(train_df)
train_df.dropna(subset=['Degerlendirme Puani'], inplace=True)
final_length = len(train_df)
print(f" - {initial_length - final_length} satır eksik hedef değişken nedeniyle kaldırıldı")

# 3. Özellik Mühendisliği
print("3. Özellik Mühendisliği")

# a. Yeni özellikler
print(" - Yeni özellikler oluşturuluyor")
# 1. Ebeveyn Eğitim Seviyesi
education_mapping = {
    'Eğitim Yok': 0,
    'İlkokul Mezunu': 1,
    'Ortaokul Mezunu': 2,
    'Lise': 3,
    'Üniversite': 4,
    'Yüksek Lisans / Doktora': 5,
    'bilinmiyor': 0,
    'Bilinmiyor': 0
}

for df in [train_df, test_df]:
    df['Anne Egitim Seviyesi'] = df['Anne Egitim Durumu'].astype(str).str.strip().map(education_mapping).fillna(0)
    df['Baba Egitim Seviyesi'] = df['Baba Egitim Durumu'].astype(str).str.strip().map(education_mapping).fillna(0)
    df['Ebeveyn Egitim Ort'] = (df['Anne Egitim Seviyesi'] + df['Baba Egitim Seviyesi']) / 2

# 2. Aile Geliri Skoru
sector_mapping = {
    'Kamu': 2,
    'Özel Sektör': 2,
    'Emekli': 1,
    'Ev Hanımı': 0,
    'Diğer': 1,
    'bilinmiyor': 0,
    'Bilinmiyor': 0,
    np.nan: 0,
    '0': 0
}

def map_working_status(status):
    if pd.isnull(status):
        return 0
    status = str(status).strip().lower()
    if status in ['evet', 'çalışıyor', 'evet çalışıyor']:
        return 1
    else:
        return 0

for df in [train_df, test_df]:
    df['Anne Calisiyor'] = df['Anne Calisma Durumu'].astype(str).apply(map_working_status)
    df['Baba Calisiyor'] = df['Baba Calisma Durumu'].astype(str).apply(map_working_status)
    df['Anne Sektor Skoru'] = df['Anne Sektor'].astype(str).str.strip().map(sector_mapping).fillna(0)
    df['Baba Sektor Skoru'] = df['Baba Sektor'].astype(str).str.strip().map(sector_mapping).fillna(0)
    df['Anne Gelir Skoru'] = df['Anne Egitim Seviyesi'] * df['Anne Sektor Skoru'] * df['Anne Calisiyor']
    df['Baba Gelir Skoru'] = df['Baba Egitim Seviyesi'] * df['Baba Sektor Skoru'] * df['Baba Calisiyor']
    df['Aile Geliri'] = df['Anne Gelir Skoru'] + df['Baba Gelir Skoru']

# 3. STK Deneyimi
print(" - STK deneyimi özelliği oluşturuluyor")
for df in [train_df, test_df]:
    df['STK Deneyimi'] = df['Aktif olarak bir STK üyesi misiniz?'] + df['Stk Projesine Katildiniz Mi?']

# 4. Metin Verilerinden Özellik Çıkarımı
print(" - Metin verilerinden özellik çıkarılıyor")
# "Girisimcilikle Ilgili Deneyiminizi Aciklayabilir misiniz?" sütunundan kelime sayısını alalım
for df in [train_df, test_df]:
    df['Girisimcilik Deneyim Kelime Sayisi'] = df['Girisimcilikle Ilgili Deneyiminizi Aciklayabilir misiniz?'].astype(str).apply(lambda x: len(x.split()))

# 5. İngilizce Seviyesi
print(" - İngilizce seviyesi kodlanıyor")
english_level_mapping = {
    'ana dilim gibi': 5,
    'çok iyi': 4,
    'iyi': 3,
    'orta': 2,
    'başlangıç': 1,
    'bilinmiyor': 0,
    'Bilinmiyor': 0,
    np.nan: 0
}

for df in [train_df, test_df]:
    df['Ingilizce Seviyesi'] = df['Ingilizce Seviyeniz?'].astype(str).str.strip().str.lower().map(english_level_mapping).fillna(0)

# 4. Modelleme
print("4. Modelleme")

# a. Hedef değişkeni ve özellikleri ayırma
print(" - Hedef değişken ve özellikler ayrılıyor")
X = train_df.drop(['Degerlendirme Puani', 'id', 'Girisimcilikle Ilgili Deneyiminizi Aciklayabilir misiniz?'], axis=1)
y = train_df['Degerlendirme Puani']

test_features = test_df.drop(['id', 'Girisimcilikle Ilgili Deneyiminizi Aciklayabilir misiniz?'], axis=1)

# b. Kategorik değişkenlerin belirlenmesi
print(" - Kategorik değişkenler belirleniyor")
# CatBoost modeline kategorik değişkenleri belirtelim
categorical_cols = X.select_dtypes(include=['object']).columns.tolist()

# c. Model Eğitimi ve Hiperparametre Optimizasyonu
print(" - Model eğitimi ve hiperparametre optimizasyonu başlıyor")

# Kullanılacak modeller ve hiperparametreler
models = {
    'CatBoost': {
        'model': CatBoostRegressor(verbose=0, random_state=42, loss_function='RMSE'),
        'params': {
            'iterations': [500, 1000],
            'depth': [4, 6, 8],
            'learning_rate': [0.01, 0.03, 0.1],
            'l2_leaf_reg': [1, 3, 5]
        }
    }
}

# Performans sonuçlarını saklamak için
model_scores = []

# K-Fold Cross-Validation
kf = KFold(n_splits=5, shuffle=True, random_state=42)

for model_name, mp in models.items():
    print(f"{model_name} modeli için eğitim yapılıyor...")
    randomized_search = RandomizedSearchCV(
        estimator=mp['model'],
        param_distributions=mp['params'],
        cv=kf,
        scoring='neg_mean_squared_error',
        n_iter=20,  # Daha geniş bir arama için iterasyon sayısını artırdık
        n_jobs=-1,
        verbose=1,
        random_state=42
    )
    randomized_search.fit(X, y, cat_features=categorical_cols)
    best_rmse = np.sqrt(-randomized_search.best_score_)
    print(f"{model_name} için en iyi RMSE: {best_rmse}")
    print(f"En iyi hiperparametreler: {randomized_search.best_params_}\n")
    model_scores.append({
        'Model': model_name,
        'Best_RMSE': best_rmse,
        'Best_Params': randomized_search.best_params_,
        'Best_Estimator': randomized_search.best_estimator_
    })

# Sonuçları DataFrame olarak saklayalım
print(" - Model karşılaştırma sonuçları hazırlanıyor")
results_df = pd.DataFrame(model_scores).sort_values(by='Best_RMSE')
print("Model Karşılaştırma Sonuçları:")
print(results_df[['Model', 'Best_RMSE']])

# En iyi modeli seçelim
best_model_info = results_df.iloc[0]
best_model = best_model_info['Best_Estimator']
print(f"\nEn iyi model: {best_model_info['Model']}")

# 5. En İyi Model ile Tahmin ve Sonuç Dosyasının Oluşturulması
print("5. En iyi model ile tahmin yapılıyor ve sonuç dosyası oluşturuluyor")

# Tüm veri üzerinde (X, y) en iyi modeli eğitelim
print(" - En iyi model tüm veri üzerinde eğitiliyor")
best_model.fit(X, y, cat_features=categorical_cols)

# Test verisi üzerinde tahmin yapalım
print(" - Test verisi üzerinde tahmin yapılıyor")
test_predictions = best_model.predict(test_features)

# Negatif tahminleri 0'a eşitleyelim (Eğer gerekiyorsa)
test_predictions = np.where(test_predictions < 0, 0, test_predictions)

submission = pd.DataFrame({
    'id': test_df['id'],
    'Degerlendirme Puani': test_predictions
})

submission.to_csv('submission.csv', index=False)
print("\nTahminler 'submission.csv' dosyasına kaydedildi.")


1. Veri Setini Yükleme
2. Veri Ön İşleme ve Temizleme
 - Yaş hesaplanıyor
 - Üniversite okuma sayısı hesaplanıyor
 - Gereksiz sütunlar çıkarılıyor
 - Binary sütunlar kodlanıyor
 - Cinsiyet sütunu kodlanıyor
 - Üniversite Türü sütunu kodlanıyor
 - Eksik değerler dolduruluyor
 - Hedef değişkende eksik değerler kontrol ediliyor
 - 1 satır eksik hedef değişken nedeniyle kaldırıldı
3. Özellik Mühendisliği
 - Yeni özellikler oluşturuluyor
 - STK deneyimi özelliği oluşturuluyor
 - Metin verilerinden özellik çıkarılıyor
 - İngilizce seviyesi kodlanıyor
4. Modelleme
 - Hedef değişken ve özellikler ayrılıyor
 - Kategorik değişkenler belirleniyor
 - Model eğitimi ve hiperparametre optimizasyonu başlıyor
CatBoost modeli için eğitim yapılıyor...
Fitting 5 folds for each of 20 candidates, totalling 100 fits
CatBoost için en iyi RMSE: 5.447921226050399
En iyi hiperparametreler: {'learning_rate': 0.1, 'l2_leaf_reg': 3, 'iterations': 1000, 'depth': 6}

 - Model karşılaştırma sonuçları hazırlanıyor
Mode